# WFiApp

Aplikacja będzie wykorzystywać `RecyclerView`, gdzie każdym elementem na liście będzie `CardView`. Dodamy również podstawową obsługę gestów oraz wykorzystamy `Intent` aby otworzyć nową aktywność w której zaprezentowany będzie bardziej szczegółowy opis wybranego elementu listy.

<table><tr><td><img src="https://media2.giphy.com/media/gooBI30gtU5fMkmCZu/giphy.gif?cid=790b7611df027999cb2a17ec28a3095ad5693e6e76c37990&rid=giphy.gif&ct=g" width="200" /></td><td><img src="https://media2.giphy.com/media/h5Xh7V5IA5E3BV4ytL/giphy.gif?cid=790b7611412fedb227b91c2cb44067aa88f517a18214afde&rid=giphy.gif&ct=g" width="150" /></td><td><img src="https://media2.giphy.com/media/uwDAUUAjLZnyKqLg3J/giphy.gif?cid=790b76110bdb83e20fb638b9fb5150422cf100e384687bcf&rid=giphy.gif&ct=g" width="150" /></td></tr></table>

W pierwszym kroku skonfigurujmy zależności `gradle`

In [ ]:
android {
    ...
    buildFeatures {
        viewBinding = true
    }
}

In [ ]:
implementation 'androidx.cardview:cardview:1.0.0'

`CardView` jest implementacją wzorca **Material Design**, który pozwala na tworzenie kart o określonym kształcie i stylu, które prezentują informacje w sposób łatwy do odczytania i przyjazny dla użytkownika. Jest to kontener, który może zawierać różne elementy, takie jak tekst, obrazy, przyciski, itp. Automatycznie przypisuje cień i tło do elementów znajdujących się w nim, co nadaje im wizualny efekt "wysunięcia" z tła.

Jest często używany w połączeniu z `RecyclerView`, aby tworzyć listy elementów w interfejsie użytkownika.

Będziemy tworzyć aplikacje w standardzie *master-detail*
- *master* - widok listy `RecyclerView`, zawierający tylko podstawowe informacje o elementach
- *detail* - widok szczegółowy elementu, zawierający pełne informacje o klikniętym elemencie

Po wybraniu elementu na fragmencie z listą (*master*), użytkownik zostanie przeniesiony do drugiego fragmentu (*detail*) - przekażemy pomiędzy fragmentami wszystkie dane dotyczące wybranego elementu. W tym celu wykorzystamy adnotację `@Parcelize`

In [ ]:
plugins {
    id 'com.android.application'
    id 'org.jetbrains.kotlin.android'
    id 'kotlin-parcelize'
}

W projekcie wykorzystamy również grafiki (zdjęcia instytrutów), kktóre umieścimy w kontenerach `ImageView` za pomocą biblioteki `Glide`

In [ ]:
implementation 'com.github.bumptech.glide:glide:4.13.0'
annotationProcessor 'com.github.bumptech.glide:compiler:4.13.0'

Biblioteka `Glide` to popularna biblioteka do ładowania obrazów. Umożliwia łatwe i wydajne pobieranie obrazów z różnych źródeł (np. z internetu, pamięci podręcznej, zasobów aplikacji) i ich wyświetlanie w `ImageView`. Obsługuje wiele formatów obrazów, w tym `JPEG`, `PNG` i `GIF`, a także pozwala na skalowanie, przycinanie, obracanie i aplikowanie filtrów do obrazów.

Dodajmy również `Jetpack Navigation`

In [ ]:
def nav_version = "2.5.3"
implementation "androidx.navigation:navigation-fragment:$nav_version"
implementation "androidx.navigation:navigation-ui:$nav_version"

Oraz plugin

In [ ]:
plugins {
    id 'com.android.application'
    id 'androidx.navigation.safeargs'
}

Następnie w pliku `build.gradle (Project)` dodajemy

In [ ]:
buildscript { // przed blokiem plugins
    repositories {
        google()
    }
    dependencies {
        def nav_version = "2.5.3"
        classpath "androidx.navigation:navigation-safe-args-gradle-plugin:$nav_version"
    }
}

## **Nawigacja**

Dodajmy dwa fragmenty - `InstituteListFragment` i `InstituteDetailFragment`

In [ ]:
class InstituteListFragment : Fragment() {
    
    private lateinit var binding: FragmentInstituteListBinding

    override fun onCreateView(
        inflater: LayoutInflater, container: ViewGroup?,
        savedInstanceState: Bundle?
    ): View {
        binding = FragmentInstituteListBinding.inflate(inflater)
        return binding.root
    }
}

In [ ]:
class InstituteDetailFragment : Fragment() {

    private lateinit var binding: FragmentInstituteDetailBinding

    override fun onCreateView(
        inflater: LayoutInflater, container: ViewGroup?,
        savedInstanceState: Bundle?
    ): View {
        binding = FragmentInstituteDetailBinding.inflate(inflater)
        return binding.root
    }
}

Dodajmy nawigację

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<navigation xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:id="@+id/navigation"
    app:startDestination="@id/instituteListFragment">

    <fragment
        android:id="@+id/instituteListFragment"
        android:name="com.example.wfiappkotlin.ui.fragments.institutelist.InstituteListFragment"
        android:label="fragment_institute_list"
        tools:layout="@layout/fragment_institute_list" >
        <action
            android:id="@+id/action_instituteListFragment_to_instituteDetailFragment"
            app:destination="@id/instituteDetailFragment" />
    </fragment>
    <fragment
        android:id="@+id/instituteDetailFragment"
        android:name="com.example.wfiappkotlin.ui.fragments.institutedetail.InstituteDetailFragment"
        android:label="fragment_institute_detail"
        tools:layout="@layout/fragment_institute_detail" />
</navigation>

Będziemy posiadać tylko jedną akcję - przejście z *master* do *detail*

### **Fragmenty**

Layout listy będzie zawierał tylko `RecyclerView`

<RelativeLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:paddingBottom="@dimen/activity_vertical_margin"
    android:paddingLeft="@dimen/activity_horizontal_margin"
    android:paddingRight="@dimen/activity_horizontal_margin"
    android:paddingTop="@dimen/activity_vertical_margin"
    android:background="#1111"
    tools:context=".MainActivity">

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<FrameLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".ui.fragments.institutelist.InstituteListFragment">

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/recyclerView"
        android:scrollbars="vertical"
        android:layout_width="match_parent"
        android:layout_height="match_parent"
        android:layout_margin="16dp"/>

</FrameLayout>

Dodajmy layout pojedynczego elementu listy `RecyclerView` - tworzymy plik **recyclerview_item** w katalogu **res/layout**

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.cardview.widget.CardView xmlns:android="http://schemas.android.com/apk/res/android"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_margin="16dp"
    app:cardBackgroundColor="#0b5294"
    app:cardCornerRadius="30dp"
    app:cardElevation="15dp">

    <RelativeLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content">

        <ImageView
            android:id="@+id/instituteImage"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:adjustViewBounds="true"
            android:contentDescription="@string/iv_desc" />

        <TextView
            android:id="@+id/title"
            style="@style/InstituteTitle"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:padding="16dp"
            android:layout_alignBottom="@id/instituteImage"
            android:theme="@style/ThemeOverlay.AppCompat.Dark"
            android:text="@string/title_placeholder" />

        <TextView
            android:id="@+id/cardTitle"
            style="@style/InstituteDetailText"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_below="@id/instituteImage"
            android:textColor="@android:color/white"
            android:padding="16dp"
            android:text="@string/title_placeholder" />

        <TextView
            android:id="@+id/subTitle"
            style="@style/InstituteDetailText"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_below="@id/cardTitle"
            android:padding="16dp"
            android:textColor="@android:color/white"
            android:text="@string/institute_placeholder" />

    </RelativeLayout>
</androidx.cardview.widget.CardView>

Następnie dodajmy layout `InstytuteDetailFragment`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<ScrollView xmlns:android="http://schemas.android.com/apk/res/android"
    android:layout_width="match_parent"
    android:layout_height="match_parent">

    <RelativeLayout xmlns:tools="http://schemas.android.com/tools"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        tools:context=".DetailActivity">

        <ImageView
            android:id="@+id/instituteImageDetail"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:adjustViewBounds="true"
            android:contentDescription="@string/iv_desc" />

        <TextView
            android:id="@+id/titleDetail"
            style="@style/InstituteDetailTitle"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_alignBottom="@id/instituteImageDetail"
            android:padding="16dp"
            android:text="@string/title_placeholder"
            android:theme="@style/ThemeOverlay.AppCompat.Dark" />

        <TextView
            android:id="@+id/universityTitleDetail"
            style="@style/InstituteDetailText"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_below="@id/instituteImageDetail"
            android:padding="16dp"
            android:text="@string/institute_placeholder"
            android:textColor="?android:textColorSecondary" />

        <TextView
            android:id="@+id/genericTextDetail"
            style="@style/InstituteDetailText"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_below="@id/universityTitleDetail"
            android:padding="16dp"
            android:text="@string/subtitle_detail_text" />

    </RelativeLayout>
</ScrollView>

Na koniec dodajmy nawiogację do layoutu głównej aktywności

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".MainActivity">

    <androidx.fragment.app.FragmentContainerView
        android:id="@+id/fragmentContainerView"
        android:name="androidx.navigation.fragment.NavHostFragment"
        android:layout_width="409dp"
        android:layout_height="729dp"
        app:defaultNavHost="true"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent"
        app:navGraph="@navigation/navigation" />
</androidx.constraintlayout.widget.ConstraintLayout>

## **Model danych**

Dodajmy klasę reprezentującą model danych

In [ ]:
data class Institute(
    val title: String, 
    val info: String, 
    val imageResource: Int // identyfikatory obrazów są przechowywane jako int
)

## **Utworzenie listy `Institute` - `DataProvider`**

Listę wszystkich informacji o instytutach (i biblioteki) będziemy przechowywać w tabelach `<array>` w pliku `strings.xml`

In [ ]:
    <string-array name="institute_titles">
        <item>Institute of Astronomy</item>
        <item>Institute of Experimental Physics</item>
        <item>Institute of Theoretical Physics</item>
        <item>Library</item>
    </string-array>

    <string-array name="institute_info">
        <item>Welcome to Institute of Astronomy!</item>
        <item>Welcome to Institute of Experimental Physics!</item>
        <item>Welcome to Institute of Theoretical Physics!</item>
        <item>Welcome to Library!</item>
    </string-array>


    <array name="institute_images">
        <item>@drawable/img_ia</item>
        <item>@drawable/img_ifd</item>
        <item>@drawable/img_ift</item>
        <item>@drawable/img_bib</item>
    </array>

Na podstawie tych informacji chcemy stworzyć kolekcję zawierającą wszystkie `Institute`. W obiekcie `DataProvider` dodajmy 

In [ ]:
val institutes: ArrayList<Institute> = ArrayList()

Dodajmy metodę `initializeData` - zainicjujemy dane w metodzie `onCreate` głównej aktywności.

In [ ]:
fun getInstituteData(activity: Activity){

Wpierw utworzymy dwie tablice `String` do przechowania danych z tablic `institute_titles` oraz `institute_info`

In [ ]:
val instituteList = activity.resources.getStringArray(R.array.institute_titles)
val instituteInfo = activity.resources.getStringArray(R.array.institute_info)

Aby przechować zasoby (odnośniki do plików graficznych z tabeli `institute_images`) skorzystamy z `TypedArray` - jest to kontener do przechowaywania wartości `Resources`.

In [ ]:
val instituteImageResources = activity.resources.obtainTypedArray(R.array.institute_images)

Następnie tworzymy listę 

In [ ]:
for (i in instituteList.indices) institutes.add(
    Institute(
        instituteList[i],
        instituteInfo[i],
        instituteImageResources.getResourceId(i, 0)
    )
)

Ostatnim elementem jest wywołanie metody `recycle` na tablicy `TypedArray`. Pozwala ona na odtworzenie tablicy, która może zostać ponownie wykorzystana.

In [ ]:
fun getInstituteData(activity: Activity){
    val instituteList = activity.resources.getStringArray(R.array.institute_titles)
    val instituteInfo = activity.resources.getStringArray(R.array.institute_info)
    val instituteImageResources = activity.resources.obtainTypedArray(R.array.institute_images)

    for (i in instituteList.indices) institutes.add(
        Institute(
            instituteList[i],
            instituteInfo[i],
            instituteImageResources.getResourceId(i, 0)
        )
    )

    instituteImageResources.recycle()
}

Podsumowując, na początku pobierane są tablice stringów i obrazów ze zasobów aplikacji przy użyciu metod `getResource().getStringArray()` oraz `getResource().obtainTypedArray()`. Następnie pętla `forEach` iteruje po wszystkich elementach tablicy `instituteList`, tworząc obiekt klasy `Institute` dla każdego elementu. Obiekt jest tworzony poprzez wywołanie konstruktora klasy `Institute` i przekazanie do niego tytułu instytutu (element z tablicy `instituteList`), informacji o instytucie (element z tablicy `instituteInfo`) oraz zasobu obrazu (wywołanie metody `getResourceId()` na obiekcie `instituteImageResources` z przekazaniem indeksu i wartości domyślnej).

Ostatecznie, po zakończeniu pętli `forEach`, zasoby obrazów są zwalniane przy użyciu metody `recycle()` na obiekcie `instituteImageResources`.

Następnie inicjujemy dane

In [ ]:
class MainActivity : AppCompatActivity() {

    private val binding: ActivityMainBinding by lazy { ActivityMainBinding.inflate(layoutInflater) }
    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        setContentView(binding.root)
        
        if (DataProvider.institutes.isEmpty())
            DataProvider.getInstituteData(this)
    }
}

## **`InstituteListAdapter`**

Kolejnym krokiem będzie utworzenie klasy `InstituteViewHolder`, `InstituteAdapter`.

In [ ]:
class InstituteViewHolder(
    private val binding: RecyclerviewItemBinding, 
    private val context: Context) : RecyclerView.ViewHolder(binding.root) {

    fun bind(institute: Institute) {
        binding.apply {
            title.text = institute.title
            subTitle.text = institute.info
            Glide.with(context).load(institute.imageResource)
                .into(instituteImage)
        }
    }
}

In [ ]:
class InstituteAdapter() : RecyclerView.Adapter<InstituteViewHolder>() {

    override fun onCreateViewHolder(parent: ViewGroup, viewType: Int): InstituteViewHolder =
        InstituteViewHolder(RecyclerviewItemBinding.inflate(
            LayoutInflater.from(parent.context), parent, false
        ), parent.context
    )

    override fun onBindViewHolder(holder: InstituteViewHolder, position: Int) {
        val currentInstitute = DataProvider.institutes[position]
        holder.bind(currentInstitute)
    }

    override fun getItemCount() = DataProvider.institutes.size
}

Mamy połączone dane tekstowe z polami `TextView`. Wykorzystujemy bibliotekę `Glide` do obsługi ładowania grafik w pola `ImageView`. `Glide.with` przyjmuje `Context` jako argument, więc przekazujemy go w konstruktorze `InstituteViewHolder`.

- `Glide.with(context)` inicjuje obiekt Glide dla bieżącego kontekstu.
- `load(institute.imageResource)` wskazuje adres zasób, który ma zostać załadowany przez `Glide`.
- `into(instituteImage)` definiuje `ImageView`, do którego ma zostać załadowany obraz.

Dodajmy `RecyclerView` do `InstituteListFragment`

In [ ]:
class InstituteListFragment : Fragment() {

    private lateinit var binding: FragmentInstituteListBinding

    override fun onCreateView(
        inflater: LayoutInflater, container: ViewGroup?,
        savedInstanceState: Bundle?
    ): View {
        binding = FragmentInstituteListBinding.inflate(inflater)

        binding.recyclerView.apply {
            adapter = InstituteAdapter()
            layoutManager = LinearLayoutManager(this@InstituteListFragment.requireContext())
        }

        return binding.root
    }
}

## **Obsługa `onClick`**

Chcemy dodać obsługę zdarzenia `onClick` elementu listy, nie będziemy implementować w metodzie `onBindViewHolder`, ponieważ chcemy uniknąć tworzenia wielu niepotrzebnych obiektów. Implementację metody przekażemyy przez konstruktor. 

In [ ]:
class InstituteViewHolder(
    private val binding: RecyclerviewItemBinding,
    private val context: Context,
    onItemClicked: (Int) -> Unit // onClick
) : RecyclerView.ViewHolder(binding.root)

Następnie w bloku `init` klasy `InstituteViewHolder` wykorzystujemy metodę `inItemClicked`

In [ ]:
init { itemView.setOnClickListener { onItemClicked(adapterPosition) } }

W klasie `InstituteAdapter` implementację metody, również przekażemy przez konstruktor - sama implementacja będzie znajdowała się w klasie `InstituteListFragment`

In [ ]:
class InstituteAdapter(private val onItemClick: (Institute) -> Unit)
    : RecyclerView.Adapter<InstituteViewHolder>() {

Metodę `onItemClicked` wywołamy w metodzie `onCreateViewHolder` - gdzie tworzymy obiekty `ViewHolder`

In [ ]:
override fun onCreateViewHolder(parent: ViewGroup, viewType: Int): InstituteViewHolder =
    InstituteViewHolder(RecyclerviewItemBinding.inflate(
        LayoutInflater.from(parent.context), parent, false
    ), parent.context
    ) { onItemClick(DataProvider.institutes[it]) }

Zanim dodamy implementację, dodajmy adnotację `@Parcelize` do modelu danych.

In [ ]:
@Parcelize
data class Institute(
    val title: String,
    val info: String,
    val imageResource: Int
) : Parcelable

Teraz dodajmy `<argument>` do akcji zdefiniowanej w pliku `navigation.xml`

In [ ]:
<action
    android:id="@+id/action_instituteListFragment_to_instituteDetailFragment"
    app:destination="@id/instituteDetailFragment" >
    <argument android:name="institute"
        app:argType="com.example.wfiappkotlin.data.Institute"/>
</action>

Samą impoplementację dodajemy przy konfiguracji `RecyclerView` w `InstituteListFragment` - jako argument konstruktora adaptera.

In [ ]:
binding.recyclerView.apply {
    adapter = InstituteAdapter() {institute ->
        val action: NavDirections = InstituteListFragmentDirections.actionInstituteListFragmentToInstituteDetailFragment(
            institute
        )
        findNavController().navigate(action)
    }
    layoutManager = LinearLayoutManager(this@InstituteListFragment.requireContext())
}

`adapter = InstituteAdapter() {institute -> ... }` przypisuje adapter typu `InstituteAdapter` do `RecyclerView`. Adapter ten obsługuje pojedynczy element listy instytutów, wykorzystując szablon widoku karty `CardView`.

`InstituteAdapter() {institute -> ... }` to lambda, która przyjmuje pojedynczy argument typu `Institute` i wywołuje kod będący obsługą zdarzenia `onClick` elementu `RecyclerView`.

`val action: NavDirections = InstituteListFragmentDirections.actionInstituteListFragmentToInstituteDetailFragment(institute)` tworzy akcję nawigacji z bieżącego fragmentu listy instytutów do fragmentu szczegółów instytutu (`InstituteDetailFragment`). Jako argument przekazujemy instytut z listy, który został kliknięty.

`findNavController().navigate(action)` uruchamia nawigację i przenosi użytkownika z fragmentu listy instytutów do fragmentu szczegółów instytutu z przekazanym argumentem.

`layoutManager = LinearLayoutManager(this@InstituteListFragment.requireContext())` ustawia menadżera układu w postaci pionowej listy, który jest wykorzystywany przez RecyclerView.

Kolejnym krokiem będzie odebranie danych we fragmencie `DetailInstituteFragment`.

In [ ]:
class InstituteDetailFragment : Fragment() {

    private lateinit var binding: FragmentInstituteDetailBinding

    override fun onCreateView(
        inflater: LayoutInflater, container: ViewGroup?,
        savedInstanceState: Bundle?
    ): View {
        binding = FragmentInstituteDetailBinding.inflate(inflater)

        setupUi(getCurrentInstitute())
        
        return binding.root
    }

    private fun getCurrentInstitute() = if (SDK_INT >= Build.VERSION_CODES.TIRAMISU)
        arguments?.getParcelable("institute", Institute::class.java)
    else
        arguments?.parcelable("institute")

    private fun setupUi(currentInstitute: Institute?) {
        Glide.with(this)
            .load(currentInstitute?.imageResource)
            .into(binding.instituteImageDetail)

        binding.titleDetail.text = currentInstitute?.title
    }
}

W pliku `StopDeprecationWarningUtil` dodaję funkcję rozszerzającą `parcelable`, zwracającą obiekt `Parcelible` dla `API` < 33.

In [ ]:
inline fun <reified T : Parcelable> Bundle.parcelable(key: String): T? = when {
    SDK_INT >= 33 -> getParcelable(key, T::class.java)
    else -> @Suppress("DEPRECATION") getParcelable(key) as? T
}

## **GridLayoutManager**

Zamiast `LinearLayoutManager` w naszym `RecyclerView`, zastosujemy `GridLayoutManager` - wykorzystamy go zmiany liczby kolumn przy zmianie orientacji ekranu. W widoku wertykalnym będziemy mieć jedną kolumnę, w horyzontalnym dwie - czyli dwa `CardView` obok siebie.

<img src="https://media2.giphy.com/media/gooBI30gtU5fMkmCZu/giphy.gif?cid=790b7611df027999cb2a17ec28a3095ad5693e6e76c37990&rid=giphy.gif&ct=g" width="200" />

Aby to osiągnąć musimy zdefiniować zmienną, która będzie zmieniała wartość po zmianie orientacji urządzenia. Do folderu **values** dodajmy nowy plik o nazwie `integers.xml`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<resources>
    <integer name="grid_column_count">1</integer>
</resources>

Chcemy teraz zdefiniować inną wartość dla zmiennej `grid_column_count` dla innej orientacji. Ponownie do folderu **values** dodajemy plik o nazwie `integers.xml` lecz tym razem w zakładce **Available qualifiers** wybieram **Orientation** i dodaję do wybranych kwalifikatorów. Następnie z rozwijanego menu wybieram **Landscape**.

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<resources>
    <integer name="grid_column_count">2</integer>
</resources>

Mechanizm ten pozwala na przygotowanie różnych wersji plików w zależności od wybranego kwalifikatora.

Powróćmy do `InstituteListFragment` i dodajmy zmienną przechowującą wartość `grid_column_count`

In [ ]:
val gridColumnCount = resources.getInteger(R.integer.grid_column_count)

Następnie ustawmy `LayoutManager`

In [ ]:
binding.recyclerView.apply {
    adapter = InstituteAdapter() {institute ->
        val action: NavDirections = InstituteListFragmentDirections.actionInstituteListFragmentToInstituteDetailFragment(
            institute
        )
        findNavController().navigate(action)
    }
    //layoutManager = LinearLayoutManager(this@InstituteListFragment.requireContext())
    layoutManager = GridLayoutManager(this@InstituteListFragment.requireContext(), gridColumnCount)
}

Konstruktor `GridLayoutManager` przyjmuje dwa parametry - `Context` oraz liczbę kolumn. Jako liczbę kolumn podajemy utworzoną `gridColumnCount`. 

<img src="https://media2.giphy.com/media/gooBI30gtU5fMkmCZu/giphy.gif?cid=790b7611df027999cb2a17ec28a3095ad5693e6e76c37990&rid=giphy.gif&ct=g" width="200" />

## **Obsługa gestów**

Dodajmy do aplikacji podstawową obsługę gestów. Chcemy mieć możliwość przestawienia elementów na liście (**drag & drop**) oraz usunięcia jednego elementu (**swipe to dismiss**). Posłużymy się klasą `ItemTouchHelper` ułatwiającą implementację odpowiedzi na zdarzania przez `RecyclerView`.

W pierwszej kolejności chcemy ustalić przesuwając element w jakich kierunkach chcemy usunąć element. Do metody `onCreate` dodajmy zmienną reprezentującą liczbę kierunków

In [ ]:
val swipeDirs = if (gridColumnCount > 1) 0 else ItemTouchHelper.LEFT or ItemTouchHelper.RIGHT

Uzależniamy możliwość usunięcia elementu listy od orientacji urządzenia - czyli od liczby wyświetlanych kolumn. Jeżeli znajdujemy się w orientacji **portrait** zmienna `swiepDirs` będzie miała wartość reprezentującą kierunki `LEFT` i `RIGHT`. Tutaj posługujemy się **alternatywą** (operator `or`).

W następnej kolejności zaimplementujemy samą funckjonalność, będziemy potrzebować instancję klasy `IteemTouchHelper`

In [ ]:
val helper = ItemTouchHelper()

Jako parametr konstruktora musimy podać `Callback` na którym będzie działać utworzony `ItemTouchHelper`. Tutaj chhcemy zaimplementować podstawową funkcjonalność, więc możemy skorzystać z uproszczonego klasy `SimpleCallback`. Jako parametr podajemy instancję klasy anonimowej 

In [ ]:
val helper = ItemTouchHelper(ItemTouchHelper.SimpleCallback{

})

Klasa `SimpleCallback` przyjmuje dwa parametry
- `dragDirs` - określający kierunki przeciągnięcia elementu
- `swipeDirs` - określający kierunki przemiecenia elementu

In [ ]:
val helper = ItemTouchHelper(object: ItemTouchHelper.SimpleCallback(
    ItemTouchHelper.LEFT or ItemTouchHelper.RIGHT or
            ItemTouchHelper.UP or ItemTouchHelper.DOWN,
    swipeDirs
)

Będziemy mieć możliwość przeciągania elementu w czterech kierunkach i wykonania **swipe to dismiss** tylko w dwóch kierunkach w orientacji wertykalnej.

Mamy dwie możliwości, więc mamy również dwie metody do zaimplementowania

In [ ]:
override fun onMove(
    recyclerView: RecyclerView,
    viewHolder: RecyclerView.ViewHolder,
    target: RecyclerView.ViewHolder
): Boolean {
}

override fun onSwiped(viewHolder: RecyclerView.ViewHolder, direction: Int) {
}

W metodzie `onSwiped` chcemy dodać implementację usuwającą przemieciony element. W pierwszej kolejności usuniemy element o zadanej pozycji z naszej kolekcji `institutes`, następnie powiadomimy `Adapter` o usuniętym elemencie aby wykonać odświeżenie `RecyclerView`

In [ ]:
override fun onSwiped(viewHolder: RecyclerView.ViewHolder, direction: Int) {
    institutes.removeAt(viewHolder.adapterPosition)
    instituteAdapter.notifyItemRemoved(viewHolder.adapterPosition)
}

Musimy podczepić utworzony `ItemTouchHelper` pod `RecyclerView`

In [ ]:
helper.attachToRecyclerView(recyclerView)

Możemy przetestować funkcjonalność **swipe to dismiss**

<img src="https://media2.giphy.com/media/h5Xh7V5IA5E3BV4ytL/giphy.gif?cid=790b7611412fedb227b91c2cb44067aa88f517a18214afde&rid=giphy.gif&ct=g" width="200" />

Dodajmy implementację metody `onMove`. Musimy zdefiniować dwie pozycje, który element przesuwamy i na które miejsce. Zdefiniujmy dwie zmienne

In [ ]:
val from = viewHolder.adapterPosition
val to = target.adapterPosition

`ViewHolder` odnosi się do elemtu na którym wykonujemy czynność, `target` jest elementem docelowym. Następnie wykonujemy metodę `swap` na naszej kolekcji i powiadamiamy `Adapter` o wykonanej operacji.

In [ ]:
override fun onMove(
    recyclerView: RecyclerView,
    viewHolder: RecyclerView.ViewHolder,
    target: RecyclerView.ViewHolder
): Boolean {
    val from = viewHolder.adapterPosition
    val to = target.adapterPosition

    Collections.swap(institutes, from, to)
    instituteAdapter.notifyItemMoved(from, to)
    return true
}

Metoda `onMove` zwraca `boolean` w zależności od powodzenia operacji. Tutaj, dla prostoty, zawsze zwrócimy `true`. Możemy przetestować funkcjonalność.

In [ ]:
class InstituteListFragment : Fragment() {

    private lateinit var binding: FragmentInstituteListBinding

    override fun onCreateView(
        inflater: LayoutInflater, container: ViewGroup?,
        savedInstanceState: Bundle?
    ): View {
        binding = FragmentInstituteListBinding.inflate(inflater)

        val gridColumnCount = resources.getInteger(R.integer.grid_column_count)
        
        setupRecyclerView(binding.recyclerView, gridColumnCount)
        val (swipeDirs, dragDirs) = getDirs(gridColumnCount)
        attachItemTouchHelperToRecyclerView(dragDirs, swipeDirs, binding.recyclerView)

        return binding.root
    }

    private fun getDirs(gridColumnCount: Int): Pair<Int, Int> {
        val swipeDirs =
            if (gridColumnCount > 1) 0 else ItemTouchHelper.LEFT or ItemTouchHelper.RIGHT
        val dragDirs = ItemTouchHelper.LEFT or ItemTouchHelper.RIGHT or
                ItemTouchHelper.UP or ItemTouchHelper.DOWN
        return Pair(swipeDirs, dragDirs)
    }

    private fun setupRecyclerView(
        recyclerView: RecyclerView,
        gridColumnCount: Int
    ) {
        recyclerView.apply {
            adapter = InstituteAdapter() { institute ->
                val action: NavDirections =
                    InstituteListFragmentDirections.actionInstituteListFragmentToInstituteDetailFragment(
                        institute
                    )
                findNavController().navigate(action)
            }
            layoutManager =
                GridLayoutManager(this@InstituteListFragment.requireContext(), gridColumnCount)
        }
    }

    private fun attachItemTouchHelperToRecyclerView(
        dragDirs: Int,
        swipeDirs: Int,
        recyclerView: RecyclerView
    ) {
        ItemTouchHelper(
            object : ItemTouchHelper.SimpleCallback(dragDirs, swipeDirs) {
                override fun onMove(
                    recyclerView: RecyclerView,
                    viewHolder: RecyclerView.ViewHolder,
                    target: RecyclerView.ViewHolder
                ): Boolean {
                    val from = viewHolder.adapterPosition
                    val to = target.adapterPosition
                    Collections.swap(institutes, from, to)
                    recyclerView.adapter?.notifyItemMoved(from, to)
                    return true
                }

                override fun onSwiped(viewHolder: RecyclerView.ViewHolder, direction: Int) {
                    institutes.removeAt(viewHolder.adapterPosition)
                    recyclerView.adapter?.notifyItemRemoved(viewHolder.adapterPosition)
                }
            }).attachToRecyclerView(recyclerView)
    }
}

<img src="https://media2.giphy.com/media/uwDAUUAjLZnyKqLg3J/giphy.gif?cid=790b76110bdb83e20fb638b9fb5150422cf100e384687bcf&rid=giphy.gif&ct=g" width="200" />